<a href="https://colab.research.google.com/github/Tejas163/Deep_Learning/blob/main/minilib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parsing MNIST

In [8]:
import gzip
import numpy as np
import struct

def parse_mnist(image_filename, label_filename):
    """ Read an images and labels file in MNIST format.  See this page:
    http://yann.lecun.com/exdb/mnist/ for a description of the file format.
    Args:
        image_filename (str): name of gzipped images file in MNIST format
        label_filename (str): name of gzipped labels file in MNIST format
    Returns:
        Tuple (X,y):
            X (numpy.ndarray[np.float32]): 2D numpy array containing the loaded
                data.  The dimensionality of the data should be
                (num_examples x input_dim) where 'input_dim' is the full
                dimension of the data, e.g., since MNIST images are 28x28, it
                will be 784.  Values should be of type np.float32, and the data
                should be normalized to have a minimum value of 0.0 and a
                maximum value of 1.0 (i.e., scale original values of 0 to 0.0
                and 255 to 1.0).
            y (numpy.ndarray[dtype=np.uint8]): 1D numpy array containing the
                labels of the examples.  Values should be of type np.uint8 and
                for MNIST will contain the values 0-9.
    """
    with gzip.open(image_filename, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows * cols)
        images = images.astype(np.float32) / 255.0

    with gzip.open(label_filename, 'rb') as f:
        magic, num_labels = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)

    return images, labels

In [9]:
images, labels = parse_mnist('/content/drive/MyDrive/10714/hw0/data/t10k-images-idx3-ubyte.gz', '/content/drive/MyDrive/10714/hw0/data/t10k-labels-idx1-ubyte.gz')
print("Images shape:", images.shape)  # Should be (60000, 784) for the training set
print("Labels shape:", labels.shape)  # Should be (60000,)

Images shape: (10000, 784)
Labels shape: (10000,)


##Implement the Softmax loss function

In [10]:
def softmax_loss(Z, y):
    """ Return softmax loss.  Note that for the purposes of this assignment,
    you don't need to worry about "nicely" scaling the numerical properties
    of the log-sum-exp computation, but can just compute this directly.
    Args:
        Z (np.ndarray[np.float32]): 2D numpy array of shape
            (batch_size, num_classes), containing the logit predictions for
            each class.
        y (np.ndarray[np.uint8]): 1D numpy array of shape (batch_size, )
            containing the true label of each example.
    Returns:
        Average softmax loss over the sample."""

    Z -= np.max(Z, axis=1, keepdims=True) #For numerical stability
    exp_scores = np.exp(Z)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    N = Z.shape[0]
    corect_logprobs = -np.log(probs[range(N),y])
    data_loss = np.sum(corect_logprobs)/N
    return data_loss

In [11]:
def test_softmax_loss():
    X,y = parse_mnist("/content/drive/MyDrive/10714/hw0/data/t10k-images-idx3-ubyte.gz",
                      "/content/drive/MyDrive/10714/hw0/data/t10k-labels-idx1-ubyte.gz")
    np.random.seed(0)

    Z = np.zeros((y.shape[0], 10))
    np.testing.assert_allclose(softmax_loss(Z,y), 2.3025850)
    Z = np.random.randn(y.shape[0], 10)
    calculated_loss = softmax_loss(Z,y)
    print(f"Calculated loss:{calculated_loss}, expected loss: 2.3025850")
    np.testing.assert_allclose(softmax_loss(Z,y), 2.7291998)

In [12]:
test_softmax_loss()

Calculated loss:2.7328716716962513, expected loss: 2.3025850


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 0.00367187
Max relative difference: 0.0013454
 x: array(2.732872)
 y: array(2.7292)

In [13]:
import numpy as np
def softmax_regression_epoch(X, y, theta, lr = 0.1, batch=100):
    """ Run a single epoch of SGD for softmax regression on the data, using
    the step size lr and specified batch size.  This function should modify the
    theta matrix in place, and you should iterate through batches in X _without_
    randomizing the order.
    Args:
        X (np.ndarray[np.float32]): 2D input array of size
            (num_examples x input_dim).
        y (np.ndarray[np.uint8]): 1D class label array of size (num_examples,)
        theta (np.ndarrray[np.float32]): 2D array of softmax regression
            parameters, of shape (input_dim, num_classes)
        lr (float): step size (learning rate) for SGD
        batch (int): size of SGD minibatch
    Returns:
        None
    """
    num_examples = X.shape[0]
    num_classes = theta.shape[1]

    for i in range(0, num_examples, batch):
        X_batch = X[i: i + batch]
        y_batch = y[i: i + batch]

        Z = X_batch @ theta
        Z -= np.max(Z, axis = 1, keepdims = True)
        exp_scores = np.exp(Z)
        probs = exp_scores / np.sum(exp_scores, axis = 1, keepdims = True)

        # Compute the gradient
        delta = probs
        delta[range(X_batch.shape[0]), y_batch] -= 1
        grad = (X_batch.T @ delta) / X_batch.shape[0]

        theta -= lr * grad


In [14]:
import numpy as np
def nn_epoch(X, y, W1, W2, lr = 0.1, batch=100):
    """ Run a single epoch of SGD for a two-layer neural network defined by the
    weights W1 and W2 (with no bias terms):
        logits = ReLU(X * W1) * W2
    The function should use the step size lr, and the specified batch size (and
    again, without randomizing the order of X).  It should modify the
    W1 and W2 matrices in place.
    Args:
        X (np.ndarray[np.float32]): 2D input array of size
            (num_examples x input_dim).
        y (np.ndarray[np.uint8]): 1D class label array of size (num_examples,)
        W1 (np.ndarray[np.float32]): 2D array of first layer weights, of shape
            (input_dim, hidden_dim)
        W2 (np.ndarray[np.float32]): 2D array of second layer weights, of shape
            (hidden_dim, num_classes)
        lr (float): step size (learning rate) for SGD
        batch (int): size of SGD minibatch
    Returns:
        None
    """
    num_examples = X.shape[0]
    hidden_dim = W1.shape[1]
    num_classes = W2.shape[1]

    for i in range(0, num_examples, batch):
        X_batch = X[i:i + batch]
        y_batch = y[i:i + batch]

        # Forward pass
        hidden_layer = np.maximum(0, X_batch @ W1)  # ReLU activation
        logits = hidden_layer @ W2

        # Softmax
        logits -= np.max(logits, axis=1, keepdims=True)
        exp_scores = np.exp(logits)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
        delta3[range(X_batch.shape[0]), y_batch] -= 1
        dW2 = (hidden_layer.T @ delta3) / X_batch.shape[0]

        delta2 = (delta3 @ W2.T) * (hidden_layer > 0) #derivative of ReLU
        dW1 = (X_batch.T @ delta2) / X_batch.shape[0]

        # Update weights
        W1 -= lr * dW1
        W2 -= lr * dW2


In [15]:
def loss_err(h,y):
    """ Helper funciton to compute both loss and error"""
    return softmax_loss(h,y), np.mean(h.argmax(axis=1) != y)

In [16]:
def train_softmax(X_tr, y_tr, X_te, y_te, epochs=10, lr=0.5, batch=100,
                  cpp=False):
    """ Example function to fully train a softmax regression classifier """
    theta = np.zeros((X_tr.shape[1], y_tr.max()+1), dtype=np.float32)
    print("| Epoch | Train Loss | Train Err | Test Loss | Test Err |")
    for epoch in range(epochs):
        if not cpp:
            softmax_regression_epoch(X_tr, y_tr, theta, lr=lr, batch=batch)
        else:
            softmax_regression_epoch_cpp(X_tr, y_tr, theta, lr=lr, batch=batch)
        train_loss, train_err = loss_err(X_tr @ theta, y_tr)
        test_loss, test_err = loss_err(X_te @ theta, y_te)
        print("|  {:>4} |    {:.5f} |   {:.5f} |   {:.5f} |  {:.5f} |"\
              .format(epoch, train_loss, train_err, test_loss, test_err))

In [17]:
X_tr, y_tr = parse_mnist("/content/drive/MyDrive/10714/hw0/data/train-images-idx3-ubyte.gz",
                         "/content/drive/MyDrive/10714/hw0/data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("/content/drive/MyDrive/10714/hw0/data/t10k-images-idx3-ubyte.gz",
                         "/content/drive/MyDrive/10714/hw0/data/t10k-labels-idx1-ubyte.gz")

train_softmax(X_tr, y_tr, X_te, y_te, epochs=10, lr=0.2, batch=100)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.35134 |   0.10182 |   0.33588 |  0.09400 |
|     1 |    0.32142 |   0.09268 |   0.31086 |  0.08730 |
|     2 |    0.30802 |   0.08795 |   0.30097 |  0.08550 |
|     3 |    0.29987 |   0.08532 |   0.29558 |  0.08370 |
|     4 |    0.29415 |   0.08323 |   0.29215 |  0.08230 |
|     5 |    0.28981 |   0.08182 |   0.28973 |  0.08090 |
|     6 |    0.28633 |   0.08085 |   0.28793 |  0.08080 |
|     7 |    0.28345 |   0.07997 |   0.28651 |  0.08040 |
|     8 |    0.28100 |   0.07923 |   0.28537 |  0.08010 |
|     9 |    0.27887 |   0.07847 |   0.28442 |  0.07970 |


In [18]:
def train_nn(X_tr, y_tr, X_te, y_te, hidden_dim = 500,
             epochs=10, lr=0.5, batch=100):
    """ Example function to train two layer neural network """
    n, k = X_tr.shape[1], y_tr.max() + 1
    np.random.seed(0)
    W1 = np.random.randn(n, hidden_dim).astype(np.float32) / np.sqrt(hidden_dim)
    W2 = np.random.randn(hidden_dim, k).astype(np.float32) / np.sqrt(k)

    print("| Epoch | Train Loss | Train Err | Test Loss | Test Err |")
    for epoch in range(epochs):
        nn_epoch(X_tr, y_tr, W1, W2, lr=lr, batch=batch)
        train_loss, train_err = loss_err(np.maximum(X_tr@W1,0)@W2, y_tr)
        test_loss, test_err = loss_err(np.maximum(X_te@W1,0)@W2, y_te)
        print("|  {:>4} |    {:.5f} |   {:.5f} |   {:.5f} |  {:.5f} |"\
              .format(epoch, train_loss, train_err, test_loss, test_err))


In [19]:
X_tr, y_tr = parse_mnist("/content/drive/MyDrive/10714/hw0/data/train-images-idx3-ubyte.gz",
                         "/content/drive/MyDrive/10714/hw0/data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("/content/drive/MyDrive/10714/hw0/data/t10k-images-idx3-ubyte.gz",
                         "/content/drive/MyDrive/10714/hw0/data/t10k-labels-idx1-ubyte.gz")
train_nn(X_tr, y_tr, X_te, y_te, hidden_dim=400, epochs=20, lr=0.2)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.15324 |   0.04697 |   0.16305 |  0.04920 |
|     1 |    0.09854 |   0.02923 |   0.11604 |  0.03660 |
|     2 |    0.07396 |   0.02143 |   0.09704 |  0.03130 |
|     3 |    0.05930 |   0.01707 |   0.08757 |  0.02860 |
|     4 |    0.04782 |   0.01315 |   0.08025 |  0.02550 |
|     5 |    0.04056 |   0.01083 |   0.07687 |  0.02400 |
|     6 |    0.03488 |   0.00913 |   0.07430 |  0.02360 |
|     7 |    0.03028 |   0.00770 |   0.07242 |  0.02290 |
|     8 |    0.02643 |   0.00637 |   0.07054 |  0.02180 |
|     9 |    0.02355 |   0.00542 |   0.06972 |  0.02140 |
|    10 |    0.02101 |   0.00465 |   0.06886 |  0.02110 |
|    11 |    0.01884 |   0.00393 |   0.06791 |  0.02070 |
|    12 |    0.01716 |   0.00322 |   0.06752 |  0.02100 |
|    13 |    0.01556 |   0.00278 |   0.06694 |  0.02090 |
|    14 |    0.01421 |   0.00237 |   0.06656 |  0.02090 |
|    15 |    0.01282 |   0.00198 |   0.06615 |  0.02090 |
|    16 |    0

In [ ]:
if __name__ == "__main__":
    X_tr, y_tr = parse_mnist("data/train-images-idx3-ubyte.gz",
                             "data/train-labels-idx1-ubyte.gz")
    X_te, y_te = parse_mnist("data/t10k-images-idx3-ubyte.gz",
                             "data/t10k-labels-idx1-ubyte.gz")

    print("Training softmax regression")
    train_softmax(X_tr, y_tr, X_te, y_te, epochs=10, lr = 0.1)

    print("\nTraining two layer neural network w/ 100 hidden units")
    train_nn(X_tr, y_tr, X_te, y_te, hidden_dim=100, epochs=20, lr = 0.2)


In [ ]:
# there are five fields within the NDArray class that you'll need to be familiar with (note that the real class member these all these fields is preceded by an underscore, e.g., _handle, _strides, etc, some of which are then exposed as a public property ... for all your code it's fine to use the internal, underscored version).
# device - A object of type BackendDevice, which is a simple wrapper that contains a link to the underlying device backend (e.g., CPU or CUDA).
# handle - A class objected that stores the underlying memory of the array. This is allocated as a class of type device.Array(), though this allocation all happens in the provided code (specifically the NDArray.make function), and you don't need to worry about calling it yourself.
# shape - A tuple specifying the size of each dimension in the array.
# strides - A tuple specifying the strides of each dimension in the array.
# offset - An integer indicating where in the underlying device.Array memory the array actually starts (it's convenient to store this so we can more easily manage pointing back to existing memory, without having to track allocations).
# Of particular importance for many of your Python implementations will be the NDArray.make call:
# def make(shape, strides=None, device=None, handle=None, offset=0):
# which creates a new NDArray with the given shape, strides, device, handle, and offset. If handle is not specified (i.e., no pre-existing memory is referenced), then the call will allocate the needed memory, but if handle is specified then no new memory is allocated, but the new NDArray points the same memory as the old one

class BackendDevice:
    def __init__(self):
        pass  # Replace with actual device initialization if needed


class NDArray:
    def __init__(self, device, handle, shape, strides, offset):
        self._device = device
        self._handle = handle
        self._shape = shape
        self._strides = strides
        self._offset = offset

    @classmethod
    def make(cls, shape, strides=None, device=None, handle=None, offset=0):
        if handle is None:
            # Allocate new memory if handle is not provided.
            # Replace with actual memory allocation based on device and shape
            # For demonstration, using a placeholder
            handle = object()

        if device is None:
            device = BackendDevice()

        if strides is None:
            # Calculate default strides if not given
            strides = tuple([1] * len(shape))

        return cls(device, handle, shape, strides, offset)

    def __repr__(self):
        return f"NDArray(shape={self._shape}, strides={self._strides}, offset={self._offset})"

    def reshape(self, new_shape):
      curr_size=1
      for dim in self._shape:
        curr_size*=dim

      new_size=1
      for dim in new_shape:
        new_size*=dim

      if curr_size!=new_size:
          raise ValueError("Product of current shape is not equal to the product of the new shape.")

      return NDArray(self._device, self._handle, new_shape, None, self._offset)  # strides will be recalculated if needed

    def permute(self, new_axes):
        new_shape = tuple(self._shape[i] for i in new_axes)
        new_strides = tuple(self._strides[i] for i in new_axes)

        return NDArray(self._device, self._handle, new_shape, new_strides, self._offset)


# Example usage
array = NDArray.make((2, 3))
print(array)


# Example usage
array = NDArray.make((2, 3))
print(array)

reshaped_array = array.reshape((3,2))
print(reshaped_array)

premute_array = array.permute((1,0))
print(premute_array)

# Creating an array referencing existing memory (demonstration)
existing_array = NDArray.make((2, 3))
new_array = NDArray.make((2, 3), handle=existing_array._handle)
new_array

NDArray(shape=(2, 3), strides=(1, 1), offset=0)
NDArray(shape=(2, 3), strides=(1, 1), offset=0)
NDArray(shape=(3, 2), strides=None, offset=0)
NDArray(shape=(3, 2), strides=(1, 1), offset=0)


NDArray(shape=(2, 3), strides=(1, 1), offset=0)

Part1

In [ ]:
# prompt: def reshape(self, new_shape):
#         """
#         Reshape the matrix without copying memory.  This will return a matrix
#         that corresponds to a reshaped array but points to the same memory as
#         the original array.
#         Raises:
#             ValueError if product of current shape is not equal to the product
#             of the new shape, or if the matrix is not compact.
#         Args:
#             new_shape (tuple): new shape of the array
#         Returns:
#             NDArray : reshaped array; this will point to thep
#         """
def reshape(self, new_shape):
        """
        Reshape the matrix without copying memory.  This will return a matrix
        that corresponds to a reshaped array but points to the same memory as
        the original array.
        Raises:
            ValueError if product of current shape is not equal to the product
            of the new shape, or if the matrix is not compact.
        Args:
            new_shape (tuple): new shape of the array
        Returns:
            NDArray : reshaped array; this will point to thep
        """
        current_size = 1
        for dim in self._shape:
            current_size *= dim

        new_size = 1
        for dim in new_shape:
            new_size *= dim

        if current_size != new_size:
            raise ValueError("Product of current shape is not equal to the product of the new shape.")

        #  Add check for compact matrix if needed
        #  if not self.is_compact():
        #    raise ValueError("The matrix is not compact.")

        return NDArray(self._device, self._handle, new_shape, None, self._offset)  # strides will be recalculated if needed


NDArray(shape=(2, 3), strides=(1, 1), offset=0)


AttributeError: 'NDArray' object has no attribute 'reshape'

Permute method

In [ ]:
# prompt: def permute(self, new_axes):
#         """
#         Permute order of the dimensions.  new_axes describes a permuation of the
#         existing axes, so e.g.:
#           - If we have an array with dimension "BHWC" then .permute((0,3,1,2))
#             would convert this to "BCHW" order.
#           - For a 2D array, .permute((1,0)) would transpose the array.
#         Like reshape, this operation should not copy memory, but achieves the
#         permuting by just adjusting the shape/strides of the array.  That is,
#         it returns a new array that has the dimensions permuted as desired, but
#         which points to the same memroy as the original array.
#         Args:
#             new_axes (tuple): permuation order of the dimensions
#         Returns:
#             NDarray : new NDArray object with permuted dimensions, pointing
#             to the same memory as the original NDArray (i.e., just shape and
#             strides changed).
#         """

def permute(self, new_axes):
        """
        Permute order of the dimensions.  new_axes describes a permuation of the
        existing axes, so e.g.:
          - If we have an array with dimension "BHWC" then .permute((0,3,1,2))
            would convert this to "BCHW" order.
          - For a 2D array, .permute((1,0)) would transpose the array.
        Like reshape, this operation should not copy memory, but achieves the
        permuting by just adjusting the shape/strides of the array.  That is,
        it returns a new array that has the dimensions permuted as desired, but
        which points to the same memroy as the original array.
        Args:
            new_axes (tuple): permuation order of the dimensions
        Returns:
            NDarray : new NDArray object with permuted dimensions, pointing
            to the same memory as the original NDArray (i.e., just shape and
            strides changed).
        """

        new_shape = tuple(self._shape[i] for i in new_axes)
        new_strides = tuple(self._strides[i] for i in new_axes)

        return NDArray(self._device, self._handle, new_shape, new_strides, self._offset)